In [31]:
import cv2
import numpy as np
import csv
import sys
import os
import math

Process images from blender and store the data (combined with the correct position data from the text file) in a csv file

In [32]:
# Function to display progress in the console
def update_progress(progress):
    bar_length = 50  # Total length of the progress bar
    filled_length = int(bar_length * progress / 100)
    bar = '#' * filled_length + '_' * (bar_length - filled_length)
    sys.stdout.write('\r[{0}] {1}%'.format(bar, progress))
    sys.stdout.flush()

Each image represents a unique possible position of the game object, in the database we describe each image with 4 values: center_pixel_x, center_pixel_y, rect_width, rect_height

In [33]:
# Function to find red regions and extract bounding rectangle
def find_bounding_rect(image):
    # Convert BGR to HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Define range of red color in HSV
    lower_red = np.array([0, 20, 25])
    upper_red = np.array([30, 255, 255])
    
    # Combine masks to get only red pixels
    red_mask = cv2.inRange(hsv, lower_red, upper_red)
    
    # Find contours
    contours, _ = cv2.findContours(red_mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    contour = max(contours, key=cv2.contourArea)
    
    # Get bounding rectangle
    x, y, w, h = cv2.boundingRect(contour)
    
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 255), 2)
    
    return (x, y, w, h), contour

In [34]:
def saharAlgorithm(contour, img):
    # Compute the center of the contour
    M = cv2.moments(contour)
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])

    # Find the point on the contour that is furthest from the center
    # It is almost always the cone top point
    # IMPORTANT: If the cone top is not visible in the image you will get undefined behavior
    distances = [np.linalg.norm(np.array([cX, cY]) - point[0]) for point in contour]
    max_distance_index = np.argmax(distances)
    furthest_point = tuple(contour[max_distance_index][0])

    X = furthest_point[0]
    Y = furthest_point[1]

    p1 = (cX, img.shape[0] - cY)
    p2 = (X, img.shape[0] - Y)

    # Difference in x coordinates
    dx = p2[0] - p1[0]

    # Difference in y coordinates
    dy = p2[1] - p1[1]

    # Angle between p1 and p2 in radians
    theta = math.atan2(dy, dx)

    angle = theta * 180 / math.pi

    if (dy < 0):
        angle = 360 + angle
    
    # Visualize
    cv2.line(img, (cX, cY), furthest_point, (255, 255, 255), 2)

    return angle

Load data from Blender

In [35]:
# Image folder path and CSV file path
images_folder = 'D:\\FRC\\Programming\\LearningProjects\\Cone_ML_PoseEstimation\\BruteForceApproach\\AllPossibleCombinations\\Images' #! Image folder is not included due to size constraints of github repository
text_file_path = '../Data/2023-Cone/PositionData.txt'
csv_file_path = '../Data/2023-Cone/FullData.csv'

In [36]:
# Count .png files in the folder
total_iterations = len([file for file in os.listdir(images_folder) if file.endswith('.jpeg')])
print(f"Total images found: {total_iterations}")
# Get a sorted list of image filenames in the folder
image_files = range(0, total_iterations)


position_data = []
with open(text_file_path, 'r') as file:
    for line in file:
        x, y, a = line.strip().split(',')
        position_data.append([float(x), float(y), float(a)])

if (len(position_data) != total_iterations):
    print(f"Warning: Number of position data points ({len(position_data)}) does not match number of images ({total_iterations}).")

Total images found: 28368


Process Data from Blender

In [37]:
targetRes = (1280, 720)

# Open CSV file for writing
with open(csv_file_path, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['Image', 'Center_X', 'Center_Y', 'Width', 'Height', 'saharAngle', 'x_position', 'y_position', 'angle'])

    # Loop through all files in the folder
    for filename in image_files:
        # Read image
        image = cv2.imread(os.path.join(images_folder, ("render_"+ str(filename)+ ".jpeg")))
        
        image = cv2.resize(image, targetRes, interpolation=cv2.INTER_LINEAR)

        # Find red rectangles
        rect, contour = find_bounding_rect(image)
        saharAngle = saharAlgorithm(contour, image)
        
        cv2.imshow("Frame", image)
        cv2.waitKey(1)
        
        x, y, w, h = rect
        center_x = x + w // 2
        center_y = y + h // 2
        csv_writer.writerow([filename, center_x, center_y, w, h, saharAngle, position_data[filename][0], position_data[filename][1], position_data[filename][2]])
        
        # Update progress
        progress = int(100 * (filename + 1) / total_iterations)
        update_progress(progress)

cv2.destroyAllWindows()

[##################################################] 100%